# ElasticsearchStore Integrations (Recommended)

## Install Packages

In [8]:
%pip install langchain
%pip install openai 
%pip install langchain-community 
%pip install python-dotenv
%pip install elasticsearch
%pip install tiktoken
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.7/906.7 kB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Import packages

In [3]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from urllib.request import urlopen
import os, json

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY2')
# elastic_cloud_id=os.getenv('ES_CLOUD_ID')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv('ES_ENDPOINT')
elastic_index_name='byte-discuss-elasticsearch-store'


## Add documents

### Let's download the sample dataset and deserialize the document.

In [5]:
import pandas as pd

# Load the CSV file
file_path = "/Users/hsb/Downloads/우리카드 데이터셋/edu_data_F.csv"
df = pd.read_csv(file_path, encoding="utf-8", index_col=0)

# Display the first 5 rows
df.head()
df.dropna(inplace=True)

df.to_dict('records')

In [28]:
# Insert the data into the Elasticsearch index
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from langchain.vectorstores import ElasticsearchStore

es = ElasticsearchStore(
	host=elastic_endpoint,
	user=elastic_user,
	password=elastic_password,
	index=elastic_index_name

es.insert_documents(df.to_dict('records'))


# Check if the data is inserted
es.search(index=elastic_index_name, body={"query": {"match_all": {}}})

TypeError: ElasticsearchStore.connect_to_elasticsearch() got an unexpected keyword argument 'elasticsearch_url'

### Split Documents into Passages

Created a chunk of size 245, which is longer than the specified 50
Created a chunk of size 288, which is longer than the specified 50
Created a chunk of size 204, which is longer than the specified 50
Created a chunk of size 281, which is longer than the specified 50
Created a chunk of size 249, which is longer than the specified 50
Created a chunk of size 285, which is longer than the specified 50
Created a chunk of size 298, which is longer than the specified 50
Created a chunk of size 270, which is longer than the specified 50
Created a chunk of size 224, which is longer than the specified 50
Created a chunk of size 288, which is longer than the specified 50
Created a chunk of size 260, which is longer than the specified 50
Created a chunk of size 199, which is longer than the specified 50
Created a chunk of size 290, which is longer than the specified 50
Created a chunk of size 251, which is longer than the specified 50
Created a chunk of size 195, which is longer than the specifie

### Index data into elasticsearch

In [5]:
# Elasticsearch 설정
elastic_host = "localhost"
elastic_port = 9200
elastic_user = "elastic"
elastic_password = "woorifisa1!"
# elastic_index_name = "your_index_name"

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# ElasticsearchStore 생성 및 문서 저장
es = ElasticsearchStore.from_documents(
    docs,
    es_url=f"http://{elastic_host}:{elastic_port}",
    index_name=elastic_index_name,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
)

es

/opt/homebrew/Caskroom/miniconda/base/envs/seminar1/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Show Result

In [14]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

## Similarity / Vector Search (Approximate KNN Search) - ApproxRetrievalStrategy()

In [17]:
# 검색 쿼리
result = es.search("코로나", top_k=5)

TypeError: Positional arguments can't be used with Elasticsearch API methods. Instead only use keyword arguments.

## Hybrid Search (Approximate KNN + Keyword Search) - ApproxRetrievalStrategy()

In [33]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


es = ElasticsearchStore.from_documents(
    docs,
    es_url=f"http://{elastic_host}:{elastic_port}",
    index_name=elastic_index_name,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True),
)

es.similarity_search("work from home policy")

[Document(metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}, page_content='This work-from-home policy will be reviewed periodically and updated as necessary, taking into account changes in public health guidance, business needs, and employee feedback.\nQuestions and Concerns'),
 Document(metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data

## Exact KNN Search (Brute Force) - ExactRetrievalStrategy()

In [24]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


es = ElasticsearchStore.from_documents(
    docs,
    es_url=f"http://{elastic_host}:{elastic_port}",
    index_name=elastic_index_name,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
    strategy=ElasticsearchStore.ExactRetrievalStrategy(),
)

es.similarity_search("work from home policy")

[Document(metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'rolePermissions': ['demo', 'manager']}, page_content='The purpose of this full-time work-from-home policy is to provide guidelines and support for employees to conduct their work remotely, ensuring the continuity and productivity of business operations during the COVID-19 pandemic and beyond.\nScope'),
 Document(metadata={'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time t

## Index / Search Documents using ELSER - SparseVectorRetrievalStrategy()

In [23]:
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


es = ElasticsearchStore.from_documents(
    docs,
    es_url=f"http://{elastic_host}:{elastic_port}",
    # index_name=elastic_index_name,
    es_user=elastic_user,
    es_password=elastic_password,
    embedding=embeddings,
    index_name=elastic_index_name + "-" + "elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(),
)

es.similarity_search("work from home policy")

Error adding texts: 116 document(s) failed to index.
First error reason: Could not find trained model [.elser_model_1]


BulkIndexError: 116 document(s) failed to index.